In [76]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Flatten , Embedding
from tensorflow import keras


In [77]:
df = pd.read_csv("/content/phrases_with_sentiments.csv")
df.sample(10)

,Phrase,Sentiment
6524,average standard typical sufficient,Neutral
2358,typical adequate average normal,Neutral
827,inspired successful happy,Positive
7111,sufficient basic normal average,Neutral
254,poor sad angry horrible,Negative
9780,sufficient typical usual average,Neutral
9182,angry failure,Negative
7422,usual average common regular,Neutral
2129,frustrated sad,Negative
8688,poor angry,Negative


In [80]:
df = df[df["Sentiment"] != "Neutral"]
df.Sentiment.value_counts()

,count
Sentiment,
Positive,3342
Negative,3243


In [81]:
df.sample(10)

,Phrase,Sentiment
439,sad weak failure,Negative
1469,good strong,Positive
4095,creative brilliant strong,Positive
826,creative strong wonderful,Positive
968,poor angry,Negative
1196,frustrated sad disappointing,Negative
3233,horrible angry bad terrible,Negative
2191,poor disappointing frustrated weak,Negative
5530,creative good brilliant,Positive
9690,angry sad bad,Negative


In [84]:
df2 = df.copy()
df2["Sentiment"] = df["Sentiment"].replace({"Positive" : 1 , "Negative" : 0})
df2.sample(10)

<ipython-input-84-23f8e8391ff5>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df2["Sentiment"] = df["Sentiment"].replace({"Positive" : 1 , "Negative" : 0})


,Phrase,Sentiment
6848,happy strong excellent brilliant,1
2728,angry bad,0
2321,strong brilliant wonderful,1
4658,frustrated weak failure poor,0
854,terrible horrible disappointing,0
8861,strong joyful excellent good,1
5330,disappointing failure,0
363,inspired happy excellent,1
1181,strong creative joyful,1
308,inspired wonderful joyful,1


In [93]:
reviews = df2["Phrase"].values
sentiments = df2["Sentiment"].values

In [115]:
hot_size = 50
encoded_rev = [one_hot(r,hot_size) for r in reviews]
for i in encoded_rev:
  if 4 in i:
    print(i)

[4, 49, 20]
[20, 28, 4, 2]
[49, 47, 23, 4]
[2, 4, 24, 20]
[23, 4]
[46, 4]
[49, 4, 46]
[49, 4, 28, 23]
[20, 4, 46, 28]
[28, 4, 23]
[20, 4, 46, 47]
[2, 46, 49, 4]
[4, 49, 24]
[4, 46, 28]
[47, 4, 49, 23]
[2, 4]
[2, 4, 49]
[4, 28]
[23, 24, 4]
[47, 4, 28]
[2, 28, 4]
[49, 4]
[4, 2]
[2, 4]
[4, 23, 28, 2]
[20, 4]
[28, 4]
[23, 20, 28, 4]
[4, 23, 2, 47]
[46, 4]
[47, 49, 4, 23]
[46, 4, 47]
[49, 2, 23, 4]
[49, 23, 4]
[4, 23, 49]
[4, 2]
[4, 49, 23, 20]
[28, 4]
[24, 23, 2, 4]
[4, 47, 49, 49]
[2, 23, 4]
[4, 46]
[46, 4]
[49, 4]
[49, 49, 4]
[49, 4, 2]
[49, 47, 24, 4]
[28, 47, 4]
[47, 4, 23]
[4, 47, 28, 24]
[4, 49]
[2, 46, 20, 4]
[4, 2]
[47, 4, 49, 2]
[24, 4, 20]
[2, 4]
[28, 24, 4, 49]
[47, 4]
[20, 47, 2, 4]
[28, 20, 24, 4]
[2, 47, 24, 4]
[4, 49, 20, 46]
[24, 20, 4]
[23, 4]
[20, 28, 49, 4]
[28, 4]
[49, 28, 4]
[4, 47]
[20, 28, 4]
[23, 49, 28, 4]
[28, 4, 2]
[4, 28, 2]
[47, 2, 4, 28]
[24, 23, 28, 4]
[28, 2, 4, 23]
[4, 49, 24]
[2, 4, 28]
[24, 4]
[28, 4, 49]
[2, 49, 4]
[23, 4, 28]
[20, 49, 49, 4]
[2, 47, 4, 

In [95]:
maxlen = 4
padded_rev = pad_sequences(encoded_rev,maxlen=maxlen,padding="post")
padded_rev

array([[47, 23, 24, 49],
       [ 4, 49, 20,  0],
       [47, 49,  0,  0],
       ...,
       [24, 23,  0,  0],
       [ 4, 24,  0,  0],
       [16, 36,  0,  0]], dtype=int32)

In [96]:
X = padded_rev
y = sentiments

In [97]:
model = keras.Sequential([
    keras.layers.Embedding(hot_size,4,input_length=maxlen,name="embedding"),
    keras.layers.Flatten(),
    keras.layers.Dense(1,activation="sigmoid")
])

model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
model.fit(X,y,epochs=50,verbose=1)
model.summary()

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


206/206 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8593 - loss: 0.5969
Epoch 2/50
206/206 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9964 - loss: 0.1888
Epoch 3/50
206/206 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9958 - loss: 0.0692
Epoch 4/50
206/206 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9949 - loss: 0.0406
Epoch 5/50
206/206 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9961 - loss: 0.0284
Epoch 6/50
206/206 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9954 - loss: 0.0235
Epoch 7/50
206/206 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9966 - loss: 0.0173
Epoch 8/50
206/206 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9969 - loss: 0.0145
Epoch 9/50
206/206 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9956 - loss: 0.0142
Epoch 10/50
206/206 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9961 - loss: 0.0131
Epoch 11/50
206/206 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9959 - loss: 0.0119
Epoch 12/50
206/206 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 4, 4)                │             200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_4 (Flatten)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 653 (2.55 KB)

 Trainable params: 217 (868.00 B)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 436 (1.71 KB)

In [103]:
w = model.get_layer("embedding").get_weights()[0]
len(w)

50

In [107]:
w[13]

array([ 0.0097182 , -0.02848343,  0.03151738, -0.03585161], dtype=float32)

In [108]:
w[26]

array([ 0.01435566, -0.04410484,  0.0484545 , -0.04291929], dtype=float32)